In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

train_df = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv')
test_df =pd.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv')



In [ ]:

train_df.head(5)





In [ ]:
test_df.head(5)

In [ ]:

import os
import numpy as np
import pandas as pd
import polars as pl
from warnings import filterwarnings
from scipy.optimize import minimize, Bounds
from gc import collect

import kaggle_evaluation.default_inference_server

filterwarnings("ignore")


MIN_POSITION = 0
MAX_POSITION = 2



In [ ]:

class UserVisibleError(Exception):
    """Custom error for invalid predictions."""
    pass


def adjusted_sharpe(solution: pd.DataFrame, submission: pd.DataFrame) -> float:
    """
    Compute a Sharpe-like score with penalties for excess volatility and poor returns.
    """

    solution = solution.copy()
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_POSITION:
        raise UserVisibleError(f"Prediction above max limit {MAX_POSITION}")
    if solution['position'].min() < MIN_POSITION:
        raise UserVisibleError(f"Prediction below min limit {MIN_POSITION}")

    # Strategy returns
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )

    excess = solution['strategy_returns'] - solution['risk_free_rate']
    cum_excess = (1 + excess).prod()
    mean_excess = cum_excess ** (1 / len(solution)) - 1
    std_excess = solution['strategy_returns'].std()

    annual_days = 252
    if std_excess == 0:
        raise ZeroDivisionError
    sharpe = mean_excess / std_excess * np.sqrt(annual_days)
    strat_vol = float(std_excess * np.sqrt(annual_days) * 100)

    # Market comparison
    market_excess = solution['forward_returns'] - solution['risk_free_rate']
    market_cum = (1 + market_excess).prod()
    market_mean = market_cum ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()
    market_vol = float(market_std * np.sqrt(annual_days) * 100)

    # Penalties
    excess_vol_penalty = (
        1 + max(0, strat_vol / market_vol - 1.2) if market_vol > 0 else 1
    )
    return_gap = max(0, (market_mean - mean_excess) * 100 * annual_days)
    return_penalty = 1 + (return_gap ** 2) / 100

    score = sharpe / (excess_vol_penalty * return_penalty)
    return float(min(score, 1_000_000))



train = pd.read_csv(
    "/kaggle/input/hull-tactical-market-prediction/train.csv",
    index_col="date_id"
)

 


In [ ]:
def objective(x):
    recent = train[-180:].copy()
    submission = pd.DataFrame({'prediction': x.clip(0, 2)}, index=recent.index)
    return -adjusted_sharpe(recent, submission)


# Initial guess + optimization
x0 = np.full(180, 0.05)
res = minimize(objective, x0, method="Powell", bounds=Bounds(0, 2), tol=1e-8)
print(res)

optimal_preds = res.x


 
counter = 0

def predict(batch: pl.DataFrame) -> float:
    global counter, optimal_preds
    value = np.float64(optimal_preds[counter])
    print(f"[{counter}] Prediction: {value:.8f}")
    counter += 1
    return value
 
server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    server.serve()
else:
    server.run_local_gateway(("/kaggle/input/hull-tactical-market-prediction/",))